In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import PIL
from cleverhans.attacks import FastGradientMethod
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import randrange
%matplotlib inline
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils import AccuracyReport
from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.attacks import LBFGS
from cleverhans.attacks import BasicIterativeMethod
from collections import Counter
from cleverhans.utils import AccuracyReport
from keras.models import load_model
batch_size = 128
num_classes = 10
epochs = 10
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
sess = tf.Session()
keras.backend.set_session(sess)
def model_arch():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model
#model = model_arch()
#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          verbose=1)
#score = model.evaluate(x_test, y_test, verbose=0)
#score[1]
def subsample(training_images,training_labels, ratio=0.8):
    shuffle_index = np.random.permutation(len(training_labels))
    training_images = training_images[shuffle_index]
    training_labels = training_labels[shuffle_index]
    sample = list()
    sample_labels = list()
    n_sample = round(training_images.shape[0] * ratio)
    while len(sample) < n_sample:
        index = randrange(training_images.shape[0])
        sample.append(training_images[index,:,:,:])
        sample_labels.append(training_labels[index])
    return np.asarray(sample),np.asarray(sample_labels)
def add_gaussian_noise(X_train, mean, stddev):
    ''' 
    INPUT:  (1) 4D numpy array: all raw training image data, of shape 
                (#imgs, #chan, #rows, #cols)
            (2) float: the mean of the Gaussian to sample noise from
            (3) float: the standard deviation of the Gaussian to sample
                noise from. Note that the range of pixel values is
                0-255; choose the standard deviation appropriately. 
    OUTPUT: (1) 4D numpy array: noisy training data, of shape
                (#imgs, #chan, #rows, #cols)
    '''
    n_imgs = X_train.shape[0]
    n_chan = X_train.shape[3]
    n_rows = X_train.shape[1]
    n_cols = X_train.shape[2]
    if stddev == 0:
        noise = np.zeros((n_imgs, n_rows, n_cols,n_chan))
    else:
        noise = np.random.normal(mean, stddev/255., 
                                 (n_imgs,n_rows, n_cols,n_chan))
    noisy_X = X_train + noise
    clipped_noisy_X = np.clip(noisy_X, 0., 1.)
    return clipped_noisy_X
def fgsm_attack(train_data,model,sess):
    wrap = KerasModelWrapper(model)
    fgsm = FastGradientMethod(wrap, sess=sess)
    fgsm_params = {'eps': 0.3,
                   'clip_min': 0.,
                   'clip_max': 1.}
    adv_x = fgsm.generate_np(train_data, **fgsm_params)
    return adv_x
def bim_attack(train_data,model,sess):
    wrap = KerasModelWrapper(model)
    bim = BasicIterativeMethod(wrap, sess=sess)
    bim_params = {'eps_iter': 0.01,
              'nb_iter': 10,
              'clip_min': 0.,
              'clip_max': 1.}
    adv_x = bim.generate_np(train_data, **bim_params)
    return adv_x
def lbfgs_attack(train_data,model,sess,tar_class):
    wrap = KerasModelWrapper(model)
    lbfgs = LBFGS(wrap,sess=sess)
    one_hot_target = np.zeros((train_data.shape[0], 10), dtype=np.float32)
    one_hot_target[:, tar_class] = 1
    adv = lbfgs.generate_np(train_data, max_iterations=10,
                                        binary_search_steps=3,
                                        initial_const=1,
                                        clip_min=-5, clip_max=5,
                                        batch_size=1, y_target=one_hot_target)
    return adv_x

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
n_ens=10
#Add gaussian noise to all images
x_noisy = add_gaussian_noise(x_train,0,64)
#variable to store the predictions of each model in the ensemble (10)
preds_ens = np.zeros((x_test.shape[0],10))
#variable to store Majority vote from all models in ensemble
max_vote_ens = np.zeros(x_test.shape[0])
for i in range(n_ens):
    #Build a new model architecture for every model in the ensemble
    model = model_arch()
    #subsample from the entire data, bagging
    sub_imgs,sub_labels = subsample(x_noisy,y_train)
    #train the model
    model.fit(sub_imgs, sub_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
    #save the model
    model.save("/Users/venkatakrishnamohansunkara/Desktop/csce 896/project/Model"+i+".h5")
    #get the predictions of the model
    ans = sess.run(tf.argmax(model.predict(x_test),axis=1))
    #store the predictions of this particular model(i) in ith column of pred_ens variable
    preds_ens[:,i]= ans.reshape((x_test.shape[0],1))
    #erase the model
    del model
#Now the variable pred_ens consists of the predictions of all test_data for each model in ensemble.
#ith column contains predictions of ith model.
#go through every row
for i in range(preds_ens.shape[0]):
    #get the entire row which consists of predictions for that particular instance from all models.
    b= Counter(preds_ens[i])
    #get the maximum vote i.e which number has more frequency.
    max_vote_ens[i] = b.most_common(1)[0][0]
#accuracy of ensemble
ens_acc = sess.run(tf.reduce_mean(tf.cast(tf.equal(max_vote_ens, tf.argmax(y_test, axis=1)) , tf.float32)))
#Build a model for normal training on the entire noisy data.
model = model_arch()
model.fit(x_noisy, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
acc = model.evaluate(x_test, y_test, verbose=0)
#accuracy of normal model on noisy train data
acc_noisy_normal = acc[1]
del model
#Build a new model for normal training (without ensemble) on entire train data (with out bagging and noise).
model = model_arch()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
acc = model.evaluate(x_test, y_test, verbose=0)
model.save("/Users/venkatakrishnamohansunkara/Desktop/csce 896/project/original_Model.h5")
#accuracy of normal model
acc_normal = acc[1]
#generate fgsm adversarial examples on test_data
adv_fgsm = fgsm_attack(x_test,model,sess)
acc_fgsm = model.evaluate(adv_fgsm, y_test, verbose=0)
#accuracy of normal model on fgsm adversarial examples
acc_fgsm = acc_fgsm[1]
#generate bim adversarial examples on test_data
adv_bim = bim_attack(x_test,model,sess)
acc_bim = model.evaluate(adv_bim,y_test,verbose=0)
#accuracy of normal model on bim adversarial examples
acc_bim = acc_bim[1]
#generate lbfgs adversarial examples on test_data
# The target is chosen as 6
adv_lbfgs = lbfgs_attack(x_test,model,sess,6)
acc_lbfgs = model.evaluate(adv_lbfgs,y_test,verbose=0)
#accuracy of normal model on lbfgs adversarial examples
acc_lbfgs = acc_lbfgs[1]
#variable to store the predictions of each model in the ensemble (10) for fgsm adversarial examples
preds_ens_fgsm = np.zeros((x_test.shape[0],10))
#variable to store Majority vote from all models in ensemble for fgsm adversarial examples
max_vote_ens_fgsm = np.zeros(x_test.shape[0])
#variable to store the predictions of each model in the ensemble (10) for bim adversarial examples
preds_ens_bim = np.zeros((x_test.shape[0],10))
#variable to store Majority vote from all models in ensemble for bim adversarial examples
max_vote_ens_bim = np.zeros(x_test.shape[0])
#variable to store the predictions of each model in the ensemble (10) for lbfgs adversarial examples
preds_ens_lbfgs = np.zeros((x_test.shape[0],10))
#variable to store Majority vote from all models in ensemble for lbfgs adversarial examples
max_vote_ens_lbfgs = np.zeros(x_test.shape[0])
for i in range(10):
    model = load_model(model.save("/Users/venkatakrishnamohansunkara/Desktop/csce 896/project/Model"+i+".h5"))
    #get predictions of model i for fgsm adversarial examples
    ans = sess.run(tf.argmax(model.predict(adv_fgsm),axis=1))
    preds_ens_fgsm[:,i]= ans.reshape((adv_fgsm.shape[0],1))
    #get predictions of model i for bim adversarial examples
    ans = sess.run(tf.argmax(model.predict(adv_bim),axis=1))
    preds_ens_bim[:,i]= ans.reshape((adv_bim.shape[0],1))
    #get predictions of model i for lbfgs adversarial examples
    ans = sess.run(tf.argmax(model.predict(adv_lbfgs),axis=1))
    preds_ens_lbfgs[:,i]= ans.reshape((adv_lbfgs.shape[0],1))
    del model
#Now the variable pred_ens consists of the predictions of all fgsm adversarial test_data for each model in ensemble.
#ith column contains predictions of ith model.
#go through every row
for i in range(preds_ens_fgsm.shape[0]):
    b= Counter(preds_ens_fgsm[i])
    max_vote_ens_fgsm[i] = b.most_common(1)[0][0]
#accuracy of ensemble on fgsm_adv
ens_acc_fgsm = sess.run(tf.reduce_mean(tf.cast(tf.equal(max_vote_ens_fgsm, tf.argmax(y_test, axis=1)) , tf.float32)))
#Now the variable pred_ens consists of the predictions of all bim adversarial test_data for each model in ensemble.
#ith column contains predictions of ith model.
#go through every row
for i in range(preds_ens_bim.shape[0]):
    b= Counter(preds_ens_bim[i])
    max_vote_ens_bim[i] = b.most_common(1)[0][0]
#accuracy of ensemble on bim_adv
ens_acc_bim = sess.run(tf.reduce_mean(tf.cast(tf.equal(max_vote_ens_bim, tf.argmax(y_test, axis=1)) , tf.float32)))
#Now the variable pred_ens consists of the predictions of all lbfgs adversarial test_data for each model in ensemble.
#ith column contains predictions of ith model.
#go through every row
for i in range(preds_ens_lbfgs.shape[0]):
    b= Counter(preds_ens_lbfgs[i])
    max_vote_ens_lbfgs[i] = b.most_common(1)[0][0]
#accuracy of ensemble on lbfgs_adv
ens_acc_lbfgs = sess.run(tf.reduce_mean(tf.cast(tf.equal(max_vote_ens_lbfgs, tf.argmax(y_test, axis=1)) , tf.float32)))
#
#-----------------------------------Adversarial Training--------------------------------------------------------------
#first adversarial examples are generated using train_data, then the model is trained on train_data+adv_train_data.
#Then the model is tested on normal test_data, then the model is tested on adversarial_test_data.
#So, we are generating the adversarial examples twice both on train and test data.
model = load_model(model.save("/Users/venkatakrishnamohansunkara/Desktop/csce 896/project/original_Model.h5"))
wrap = KerasModelWrapper(model)
#generate adversarial examples on train data.
adv_fgsm_train = fgsm_attack(x_train,model,sess)
adv_bim_train = bim_attack(x_train,model,sess)
adv_lbfgs_train = lbfgs_attack(x_train,model,sess)
train_plus_adv_fgsm = np.concatenate([x_train,adv_fgsm_train])
y_train_plus_adv_fgsm = np.concatenate([y_train,y_train])
train_plus_adv_bim = np.concatenate([x_train,adv_bim_train])
y_train_plus_adv_bim = np.concatenate([y_train,y_train])
train_plus_adv_lbfgs = np.concatenate([x_train,adv_lbfgs_train])
y_train_plus_adv_lbfgs = np.concatenate([y_train,y_train])
del model
#build a fresh model for fgsm training
model = model_arch()
wrap = KerasModelWrapper(model)
model.fit(train_plus_adv_fgsm, y_train_plus_adv_fgsm,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
fgsm_acc_train = model.evaluate(x_test,y_test,verbose=0)
#Accuracy of adversarially trained model on clean examples
fgsm_acc_train[1]
#generate adversarial examples for adversarially trained model on test_data
adv_fgsm_test = fgsm_attack(x_test,model,sess,6)
fgsm_adv_acc_train = model.evaluate(adv_fgsm_test,y_test,verbose=0)
#Accuracy of adversarially trained model on adv_test images
fgsm_adv_acc_train[1]
del model
#build a fresh model for bim training
model = model_arch()
wrap = KerasModelWrapper(model)
model.fit(train_plus_adv_bim, y_train_plus_adv_bim,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
bim_acc_train = model.evaluate(x_test,y_test,verbose=0)
#Accuracy of adversarially trained model on clean examples
bim_acc_train[1]
#generate adversarial examples for adversarially trained model on test_data
adv_bim_test = bim_attack(x_test,model,sess,6)
bim_adv_acc_train = model.evaluate(adv_bim_test,y_test,verbose=0)
#Accuracy of adversarially trained model on adv_test images
bim_adv_acc_train[1]
del model
#build a fresh model for lbfgs training
model = model_arch()
wrap = KerasModelWrapper(model)
model.fit(train_plus_adv_lbfgs, y_train_plus_adv_lbfgs,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
lbfgs_acc_train = model.evaluate(x_test,y_test,verbose=0)
#Accuracy of adversarially trained model on clean examples
lbfgs_acc_train[1]
adv_lbfgs_test = lbfgs_attack(x_test,model,sess,6)
lbfgs_adv_acc_train = model.evaluate(adv_lbfgs_test,y_test,verbose=0)
#Accuracy of adversarially trained model on adv_test images
lbfgs_adv_acc_train[1]